In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

libraries we will use:

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt # visualizing data
import seaborn as sns 
from collections import Counter
%matplotlib inline
 
from plotly.offline import init_notebook_mode, iplot
 
import os
print(os.listdir("../input"))
 
import plotly.graph_objs as go
import seaborn as sns
from sklearn.preprocessing import StandardScaler

reading the data


In [ ]:
df=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
              

showing the format of the data will helpe us when dealing with it

In [ ]:
df.head(10)


here the some information about our data ........shape, info, columns name

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

here we will show the percentage of males and females who interactive more with Response 

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,7))
plt.pie(df.groupby('Gender')['Response'].sum(),labels=('Female','Male'), explode = [0.1,0.1],autopct ='%1.1f%%',shadow = True,startangle = 180)
plt.legend()
plt.show()


In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,7))
ax1=sns.barplot(df['Gender'],'Response',data=df)

now we will display the number of females and males which have Driving_License

In [ ]:
ax=sns.countplot(df['Driving_License'],hue='Gender',data=df)

now we will show that the number of males and females which have Driving_License and responsed in the the same time

In [ ]:
ax=sns.barplot(df['Driving_License'],'Response',hue='Gender',data=df)

here we will display how many regions we interact with 

In [ ]:
 df['Region_Code'].value_counts()

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,7))
df.groupby('Region_Code')['Response'].count().nlargest(10).sort_values().plot(kind='barh')

we find that the regions which have number (25 and 8) have more peoples which response more ,so the company will intersted in 

In [ ]:
df['Vehicle_Age'].unique()

In [ ]:
sns.barplot(df['Vehicle_Age'],y='Response',data=df)

In [ ]:
sns.barplot(df['Vehicle_Damage'],'Response',hue='Gender',data=df)

In [ ]:
df['Policy_Sales_Channel'].unique()

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,7))
df.groupby('Policy_Sales_Channel')['Response'].count().nlargest(10).sort_values().plot(kind='barh')

as we notice last diagram ,the company should concentrate on  'Policy_Sales_Channel' =125

In [ ]:
sns.countplot(df['Previously_Insured'],hue="Gender",data=df)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,7))
ax1=sns.barplot(df['Previously_Insured'],'Response',data=df)
plt.show()

we notice that all only people who have no  previously_indured is responsed to new response

In [ ]:
df.head()

In [ ]:
#correlation matrix
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 12))
#sns.heatmap(corrmat, vmax=.8, square=True);
sns.heatmap(corrmat,fmt=".2f",cmap='coolwarm', annot=True)

we find two catogrical  features so we convert them to numerical feauters to deel with 

In [ ]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

df['Vehicle_Age']=df['Vehicle_Age'].replace({'> 2 Years':0,'1-2 Year':1,'< 1 Year':3})
df['Vehicle_Damage']=df['Vehicle_Damage'].replace({'Yes':5,'No':6})

In [ ]:
df.head()

before we build our classification model we will drop some features which don't help us in our algorithm such as (vintage, id) which have zero correlation with response

In [ ]:
df=df.drop(['id','Vintage'],axis=1)
test=test.drop(['id','Vintage'],axis=1)

then we will define the X and y to build our classification model

In [ ]:
X=df.drop(['Response'],axis=1)
y=df['Response']

In [ ]:
#Standard Scaler for Data

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X = scaler.fit_transform(X)


we will split our train data into trian and test to tunning hyperparamaters then launch our model to our test data to get best accuracy 

In [ ]:
#Import Libraries
from sklearn.model_selection import train_test_split
#----------------------------------------------------

#----------------------------------------------------
#Splitting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=44, shuffle =True)

#Splitted Data
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

let's try SVM MODEL

In [ ]:
#Import Libraries
from sklearn.svm import SVC
#----------------------------------------------------

#----------------------------------------------------
#Applying SVC Model 

'''
sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True,
                probability=False, tol=0.001, cache_size=200, class_weight=None,verbose=False,
                max_iter=-1, decision_function_shape='ovr’, random_state=None)
'''

SVCModel = SVC(kernel= 'linear',# it can be also linear,poly,sigmoid,precomputed
               max_iter=2000,C=1.0,gamma='auto')
SVCModel.fit(X_train, y_train)

#Calculating Details
print('SVCModel Train Score is : ' , SVCModel.score(X_train, y_train))
print('SVCModel Test Score is : ' , SVCModel.score(X_test, y_test))
#print('----------------------------------------------------')

#Calculating Prediction
y_pred = SVCModel.predict(X_test)
#print('Predicted Value for SVCModel is : ' , y_pred[:10])

let's RandomForestClassifier 

In [ ]:
#Import Libraries
from sklearn.ensemble import RandomForestClassifier
#----------------------------------------------------

#----------------------------------------------------
#Applying RandomForestClassifier Model 

'''
ensemble.RandomForestClassifier(n_estimators='warn’, criterion=’gini’, max_depth=None,
                                min_samples_split=2, min_samples_leaf=1,min_weight_fraction_leaf=0.0,
                                max_features='auto’,max_leaf_nodes=None,min_impurity_decrease=0.0,
                                min_impurity_split=None, bootstrap=True,oob_score=False, n_jobs=None,
                                random_state=None, verbose=0,warm_start=False, class_weight=None)
'''

RandomForestClassifierModel = RandomForestClassifier(criterion = 'entropy',n_estimators=500,max_depth=50,random_state=33) #criterion can be also : entropy 
RandomForestClassifierModel.fit(X_train, y_train)

#Calculating Details
print('RandomForestClassifierModel Train Score is : ' , RandomForestClassifierModel.score(X_train, y_train))
print('RandomForestClassifierModel Test Score is : ' , RandomForestClassifierModel.score(X_test, y_test))
#print('RandomForestClassifierModel features importances are : ' , RandomForestClassifierModel.feature_importances_)
#print('----------------------------------------------------')



In [ ]:
#Calculating Prediction
y_pred = RandomForestClassifierModel.predict(X_test)
y_pred_prob = RandomForestClassifierModel.predict_proba(X_test)
#print('Predicted Value for RandomForestClassifierModel is : ' , y_pred[:10])
#print('Prediction Probabilities Value for RandomForestClassifierModel is : ' , y_pred_prob[:10])

In [ ]:
#Import Libraries
from sklearn.metrics import roc_auc_score
#----------------------------------------------------

#----------------------------------------------------
#Calculating ROC AUC Score:  
#roc_auc_score(y_true, y_score, average=’macro’, sample_weight=None,max_fpr=None)

ROCAUCScore = roc_auc_score(y_test,y_pred, average='micro') #it can be : macro,weighted,samples
print('ROCAUC Score : ', ROCAUCScore)

In [ ]:
test['Gender']=test['Gender'].replace({'Male':0,'Female':1})

test['Vehicle_Age']=test['Vehicle_Age'].replace({'> 2 Years':0,'1-2 Year':1,'< 1 Year':3})
test['Vehicle_Damage']=test['Vehicle_Damage'].replace({'Yes':5,'No':6})

In [ ]:
#Calculating Prediction
y_pred = RandomForestClassifierModel.predict(test)
y_pred_prob = RandomForestClassifierModel.predict_proba(test)
print('Predicted Value for RandomForestClassifierModel is : ' , y_pred[:10])
print('Prediction Probabilities Value for RandomForestClassifierModel is : ' , y_pred_prob[:10])